### EarlyStopping and ModelCheckpoint

In [1]:
import keras
from keras import Sequential

callbacks_list = [                              
    keras.callbacks.EarlyStopping(              
        monitor='acc',                          
        patience=1,                             
    ),
    keras.callbacks.ModelCheckpoint(            
        filepath='my_model.h5',                 
        monitor='val_loss',                     
        save_best_only=True,                    
    )
]


#model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])                  
#model.fit(x, y,epochs=10,batch_size=32,callbacks=callbacks_list,validation_data=(x_val, y_val))       

/home/farid/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### ReduceLROnPlateau

In [2]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',                  
        factor=0.1,                         
        patience=10,                        
    )
]

#model.fit(x, y,epochs=10,batch_size=32,callbacks=callbacks_list,validation_data=(x_val, y_val))   4

### Writing your own callback

In [3]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):

    def set_model(self, model):
        self.model = model                                               
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input,
                                                    layer_outputs)       

    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')

        validation_sample = self.validation_data[0][0:1]                 
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'w')     
        np.savez(f, activations)                                         
        f.close()         

### TensorBoard

In [4]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000                                              
max_len = 500                                                    

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainab

In [5]:
!mkdir my_log_dir

mkdir: cannot create directory ‘my_log_dir’: File exists


In [6]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',                  
        histogram_freq=1,                      
        embeddings_freq=1,                     
    )
]
history = model.fit(x_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 88s 4ms/step - loss: 0.6364 - acc: 0.6494 - val_loss: 0.4192 - val_acc: 0.8328
Epoch 2/5
20000/20000 [==============================] - 90s 5ms/step - loss: 0.4303 - acc: 0.8191 - val_loss: 0.4358 - val_acc: 0.8172
Epoch 3/5
20000/20000 [==============================] - 97s 5ms/step - loss: 0.3704 - acc: 0.8039 - val_loss: 0.4846 - val_acc: 0.7786
Epoch 4/5
20000/20000 [==============================] - 95s 5ms/step - loss: 0.3538 - acc: 0.7629 - val_loss: 0.8551 - val_acc: 0.6322
Epoch 5/5
20000/20000 [==============================] - 92s 5ms/step - loss: 0.2858 - acc: 0.7829 - val_loss: 0.5179 - val_acc: 0.7328


In [10]:
from keras.utils import plot_model
# sudo apt-get install -y graphviz libgraphviz-dev
plot_model(model, show_shapes=True, to_file='model1.png')